In [ ]:
import os
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import numpy as np
import re

def evaluate_sql(client, question, answer_sql_expected, answer_sql_LLM, model="gpt-4o-mini"):
    content = f"""I have one question and two sql statements describing the query for that question.
One sql statement is a response from LLM and the other is a human curated sql statement.

1. Question: {question}

2. Human curated sql query: {answer_sql_expected}

3. Response sql query from LLM: {answer_sql_LLM}

Evaluate whether the LLM generated SQL query is correct or not based on the example sql query from human.
Only answer using True or False and do not give the reason.
"""
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
    model=model,
    )

    response_text = chat_completion.choices[0].message.content
    assert response_text in ["True", "False"]
    if response_text == "True":
        response = True
    elif response_text == "False":
        response = False
    
    return response, chat_completion

def evaluate_answer(client, question, sql, answer_summary_expected, answer_summary_LLM, model="gpt-4o-mini"):
    content = f"""I have one question, one SQL query, and two statements describing the query result.
One statement is a response from LLM, the other is a human curated answer.

1. Question: {question}

2. SQL query: {sql}

3. Human curated answer: {answer_summary_expected}

4. Response from LLM: {answer_summary_LLM}

Rate the LLM response on a scale of 1 to 5, where:

1. Completely Incorrect
2. Mostly Incorrect
3. Partially Correct
4. Mostly Correct
5. Completely Correct

Start the answer with the rating then provide a detailed explanation for your rating.
An example answer would be: 
Rating 5 (Completely Correct). The answer is correct because ..."""
    
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
    model=model,
    )

    # Extract the rating number using a regular expression
    response_text = chat_completion.choices[0].message.content
    rating_match = re.search(r'\b[1-5]\b', response_text)
    
    if rating_match:
        rating = int(rating_match.group(0))
    else:
        rating = None
        
    return rating, response_text, chat_completion

In [ ]:
config = load_dotenv("../.env")  

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

df = pd.read_csv("1.Refined_benchmark_response.tsv", sep="\t", index_col=0)
df_subset = df.copy()

df_evaluation = df.copy()
df_evaluation["summary_evaluation"] = None
df_evaluation["sql_evaluation"] = None

In [ ]:
for index in df_subset.index:
    if df_subset.loc[index, "sql_query_success"]:
        model = df_subset.loc[index, "model"]
        question = df_subset.loc[index, "question"]
        answer_sql_expected = df_subset.loc[index, "answer_sql_expected"]
        answer_sql_LLM = df_subset.loc[index, "answer_sql_LLM"]
        answer_summary_expected = df_subset.loc[index, "answer_summary_expected"]
        answer_summary_LLM = df.loc[index, "answer_summary_LLM"]
        if "summary_evaluation" in df_evaluation.columns:
            if type(df_evaluation.loc[index, "summary_evaluation"]) is int:
                pass
            else:
                print("Evaluating answer:", index)
                rating, evaluation_message, evaluation = evaluate_answer(client, question, answer_sql_expected, answer_summary_expected, answer_summary_LLM)
                df_evaluation.loc[index, "evaluator_model"] = model
                df_evaluation.loc[index, "summary_evaluation_reason"] = evaluation_message
                df_evaluation.loc[index, "summary_evaluation"] = rating
        if "sql_evaluation" in df_evaluation.columns:
            if type(df_evaluation.loc[index, "sql_evaluation"]) is bool:
                pass
            else:
                print("Evaluating sql:", index)
                sql_evaluation_result, sql_evaluation = evaluate_sql(client, question, answer_sql_expected, answer_sql_LLM)
                df_evaluation.loc[index, "sql_evaluator_model"] = model
                df_evaluation.loc[index, "sql_evaluation"] = sql_evaluation_result